In [1]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl

In [2]:
df = pd.read_csv('./Data/scrapeddraft0522.csv',index_col=0)

In [3]:
df = df[df.player != '<th aria-label="Player" class="sort_default_asc show_partial_when_sorting left" data-stat="player">Player</th>']

In [4]:
df.player[df.player.str.contains('=')] = df.player[df.player.str.contains('=')].apply(lambda x: str(x).split(">")[1].split("<")[0])

In [5]:
df[['team','round','pick']] = df['draft_info'].apply(lambda x: pd.Series(str(x).split('/')[0:3]))

In [6]:
df['round'] = df['round'].apply(lambda x: pd.Series(str(x).strip()))

In [7]:
df = df[df['round'] != '']

In [8]:
df['round'] = df['round'].apply(lambda x: pd.Series(str(x)[0]))

In [9]:
df['round'][df['round']=='n'] = 8

C:\Users\capta\AppData\Local\Temp\ipykernel_13772\1221531417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round'][df['round']=='n'] = 8


In [10]:
df['pick'] = df['pick'].apply(lambda x: pd.Series(str(x).strip().split(' ')[0]))

In [11]:
df['pick'] = df['pick'].apply(lambda x: pd.Series(str(x)[0:-2]))

In [12]:
df['pick'][df['pick'] == ''] = np.nan

C:\Users\capta\AppData\Local\Temp\ipykernel_13772\1104288695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pick'][df['pick'] == ''] = np.nan


In [13]:
df[['feet','inches']] = df['height'].apply(lambda x: pd.Series(str(x).split('-')))

In [14]:
df['feet'] = pd.to_numeric(df.feet,errors='coerce')
df['inches'] = pd.to_numeric(df.inches,errors='coerce')
df['height'] = (df.feet * 12) + df.inches

In [15]:
df.drop(['inches','feet'],axis=1,inplace=True)

In [16]:
df.drop(['link','draft_info'],axis = 1, inplace=True)

In [17]:
# Call NFL Players Python pacakge
player = nfl.import_players()

# Find all players who were drafted in 2005 or later
draft_players = player.loc[(player.entry_year > 2004) & (player.draft_number >= 1)].sort_values(by=['entry_year','draft_number'])

# Select relevant columns of draft players
draft_player = draft_players[['display_name','position_group','entry_year','college_conference','draft_number']]

In [18]:
cat_cols = ['player','pos','school_name','team']

In [19]:
num_cols = [col for col in df if col not in cat_cols]

In [20]:
df[num_cols] = df[num_cols].apply(pd.to_numeric,errors='coerce', axis=1)

In [21]:
drafted = df[df['round'] != 8]

In [22]:
draft = df.merge(draft_player,how='left',left_on=['year','pick'],right_on=['entry_year','draft_number'])

In [23]:
age = pd.read_excel('./Data/Age.xlsx')

In [24]:
age = age[['Year','Rnd','Pick','Age','Player']]

In [25]:
age[['Year','Rnd','Pick','Age']] = age[['Year','Rnd','Pick','Age']].apply(pd.to_numeric,errors='coerce',axis=1)

In [26]:
drafts = draft.merge(age,how='left',left_on=['year','pick','player'],right_on=['Year','Pick','Player'])

In [27]:
drafts.drop(['display_name','position_group','entry_year','draft_number','Year','Rnd','Pick','Player'],axis=1,inplace=True)

In [28]:
drafts.pos[(drafts.pos == 'OLB')|(drafts.pos == 'ILB')] = 'LB'
drafts.pos[(drafts.pos == 'OT')|(drafts.pos == 'C')|(drafts.pos == 'OG')] = 'OL'

C:\Users\capta\AppData\Local\Temp\ipykernel_13772\1009202155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.pos[(drafts.pos == 'OLB')|(drafts.pos == 'ILB')] = 'LB'
C:\Users\capta\AppData\Local\Temp\ipykernel_13772\1009202155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.pos[(drafts.pos == 'OT')|(drafts.pos == 'C')|(drafts.pos == 'OG')] = 'OL'


In [29]:
drafts['Round'] = drafts['round']

In [30]:
drafts.drop(['round'],axis=1,inplace=True)

In [31]:
drafts.college_conference[(drafts.college_conference == 'NO COLLEGE') & (drafts.school_name == 'Ohio State')] = 'Big Ten Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_13772\1151124999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO COLLEGE') & (drafts.school_name == 'Ohio State')] = 'Big Ten Conference'


In [32]:
drafts.college_conference[(drafts.college_conference == 'NO COLLEGE')] = 'Atlantic Coast Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_13772\3583540359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO COLLEGE')] = 'Atlantic Coast Conference'


In [33]:
drafts.college_conference[(drafts.college_conference == 'NO FOOTBALL')] = 'Conference USA'

C:\Users\capta\AppData\Local\Temp\ipykernel_13772\2083932286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO FOOTBALL')] = 'Conference USA'


In [34]:
drafts.college_conference[(drafts.college_conference == 'Pacific Ten Conference')] = 'Pacific Twelve Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_13772\3775298207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'Pacific Ten Conference')] = 'Pacific Twelve Conference'


In [38]:
drafts['speed'] = (drafts.weight * 200) / (drafts.forty_yd ** 4)
drafts['agility'] = (drafts.weight * 200) / (drafts.shuttle ** 4)
drafts['alt_agility'] = (drafts.weight * 200) / (drafts.cone ** 3)

In [39]:
drafts.to_parquet('./Data/draft0522.parquet')

In [37]:
#drafts.school_name[(drafts.college_conference.isna())].value_counts()